## Imports

In [32]:
# Imports for all modules needed.
import mysql.connector
import sys
import json
from datetime import datetime
import pandas as pd
sys.path.append('../DataGeneration')
from license_data_generator import License, Corrupt

## Database Setup

In [33]:
# Connect to mySQL.
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password"
)

mycursor = mydb.cursor()

# Create a database for license data if it doesn't already exist.
try:
    mycursor.execute("CREATE DATABASE licensedata")
except:
   pass

# List all available databases.
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('information_schema',)
('licensedata',)
('mysql',)
('performance_schema',)
('sys',)


## Normal License Data

In [34]:
# Creating a normal dataset.
test_dataset = License.generate_dataset(100)
test_dataset.head()

,First Name,Last Name,Date of Birth,Place of Birth,Gender,Date of Issue,Date of Expiry,Issuing Authority,License Number,Address
0,Mitchell,Saunders,09.04.1978,Puerto Rico,Male,22.12.2007,19.12.2017,DA1,SAUND704098M99KK,"429 Glenn light, Manningfort, CF4 2NN"
1,Jennifer,Davis,26.11.1955,Austria,Male,25.11.2012,23.11.2022,DA1,DAVIS511265J99PI,"329 Walsh way, South Rachaelhaven, DN2 3AW"
2,Jeffrey,Ryan,02.07.1962,Macao,Female,21.04.1993,19.04.2003,DA1,RYAN9657022J99FX,"417 Alison plain, Thomastown, KW5 7UX"
3,Joanne,Hussain,01.09.1973,Mauritius,Male,29.08.2021,27.08.2031,DA1,HUSSA709013J99QI,"342 Eric field, Liamside, JE5 7DT"
4,Joan,Williams,03.04.1959,Switzerland,Male,13.11.1979,10.11.1989,DA1,WILLI504039J99XE,"558 Marie locks, Pottsside, W5 0XX"


In [35]:
# Connect to mySQL database.
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="licensedata"
)
mycursor = mydb.cursor()

# Create a table in MySQL (if it doesn't exist) based on the columns in the dataframe. 
table_name = 'test_license_dataset'
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (id INT AUTO_INCREMENT PRIMARY KEY, "

# Extract column names from DataFrame and generate SQL query with backticks for column names to avoid conflicts with variable names.
for column in test_dataset.columns:
    column_name = column
    create_table_query += f"`{column_name}` VARCHAR(255), "

# Complete the SQL query and create the table.
try:
    create_table_query = create_table_query.rstrip(', ') + ");"
    mycursor.execute(create_table_query)
    mydb.commit()
except:
    pass

# Insert DataFrame data into the MySQL table.
insert_query = f"INSERT INTO {table_name} ("
insert_query += ", ".join([f"`{col}`" for col in test_dataset.columns]) + ") VALUES ("
insert_query += ", ".join(["%s" for _ in range(len(test_dataset.columns))]) + ")"

# Iterate through DataFrame rows to insert data.
try:
    for _, row in test_dataset.iterrows():
        mycursor.execute(insert_query, tuple(row))
except:
    pass

mydb.commit()

## Corrupt License Data

In [36]:
# Creating a dataset with corrupt entries.
corrupted_test_dataset = Corrupt.introduce_corruptions(License.generate_dataset(100), 0.1)
corrupted_test_dataset.head()

,First Name,Last Name,Date of Birth,Place of Birth,Gender,Date of Issue,Date of Expiry,Issuing Authority,License Number,Address
0,=6ron,Potts,23.08.2005,Dominica,Male,18.02.2023,15.02.2033,DA1,POTTS008235A99OT,"622 Clive route, Williamsside, RG8 5HL"
1,Eileen,H6rvey,21.11.1961,Britis? (ndian|Ocean Territory (Ch:gos Archipe...,Female,01.01.2009,30.12.2018,DA1,HARVE661211E99HX,"461 Brown lakes, North Brenda, PR4 3TJ"
2,Mandy,|arr:s,60.18.1959,"""ruba",Male,26.08.1998,23.08.2008,DA1,HARRI504169M99YH,"927 Harriet trace, Port Chloe, M8 2QE"
3,Graham,Richa@dson,20.08.1952,South Africa,Male,30.10.1981,28.10.1991,DA1,RICHA508202G99IY,"201 Jackson well, Marshville, G48 1FU"
4,Jay,Parkes,27/03/1984,"Falkland Is""ands ((alvinas)",Male,24.02.2020,21.02.2030,DA1,PARKE803274J99QS,"211 Guy road, Port Chloe, G3 9SB"


In [37]:
# Connect to mySQL database.
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="licensedata"
)
mycursor = mydb.cursor()

# Create a table in MySQL (if it doesn't exist) based on the columns in the dataframe. 
table_name = 'corrupt_test_license_dataset'
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (id INT AUTO_INCREMENT PRIMARY KEY, "

# Extract column names from DataFrame and generate SQL query with backticks for column names to avoid conflicts with variable names.
for column in corrupted_test_dataset.columns:
    column_name = column
    create_table_query += f"`{column_name}` VARCHAR(255), "

# Complete the SQL query and create the table.
try:
    create_table_query = create_table_query.rstrip(', ') + ");"
    mycursor.execute(create_table_query)
    mydb.commit()
except:
    pass

# Insert DataFrame data into the MySQL table.
insert_query = f"INSERT INTO {table_name} ("
insert_query += ", ".join([f"`{col}`" for col in corrupted_test_dataset.columns]) + ") VALUES ("
insert_query += ", ".join(["%s" for _ in range(len(corrupted_test_dataset.columns))]) + ")"

# Iterate through DataFrame rows to insert data.
try:
    for _, row in corrupted_test_dataset.iterrows():
        mycursor.execute(insert_query, tuple(row))
except:
    pass

mydb.commit()

## Summary Table Creation

In [38]:
# Connect to mySQL database.
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="licensedata"
)
mycursor = mydb.cursor()

# Create a table to store general information and the metadata about each dataset.
metadata_table_query = """
    CREATE TABLE IF NOT EXISTS dataset_registry (
        table_id INT AUTO_INCREMENT PRIMARY KEY,
        table_name VARCHAR(255),
        num_rows INT,
        num_columns INT,
        metadata JSON
    )
"""

try:
    mycursor.execute(metadata_table_query)
    mydb.commit()
except:
    pass

In [39]:
dataset_metadata = {   
    # Metadata for the test dataset.
    'test_license_dataset' : {
        'name': 'English',
        'description': 'This dataset contains synthetic data for driver license data.',
        'language': 'English',
        'creation_date': datetime.now().strftime("%d.%m.%y %H:%M:%S"),
        'dataframe_size': test_dataset.shape,
        'columns': test_dataset.columns.tolist(),
        'corruption': 'Needs to be created',
        'num_corrupted_entries': 'Needs to be created',
        'dataset_source': 'https://github.com/AatishDA1/PracticeSDProject.git'
    },

    # Metadata for the corrupt test dataset.
    'corrupt_test_license_dataset' : {
        'description': 'This dataset contains synthetic data for driver license data, with some corruptions.',
        'language': 'English',
        'creation_date': datetime.now().strftime("%d.%m.%y %H:%M:%S"),
        'dataframe_size': test_dataset.shape,
        'columns': test_dataset.columns.tolist(),
        'corruption': 'Needs to be created',
        'num_corrupted_entries': 'Needs to be created',
        'dataset_source': 'https://github.com/AatishDA1/PracticeSDProject.git'
    }
}

In [40]:
# Connect to mySQL database.
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="licensedata"
)
mycursor = mydb.cursor()

# Fetch all tables in the database.
all_tables_query = "SHOW TABLES"
mycursor.execute(all_tables_query)
all_tables = mycursor.fetchall()

# Filter out the 'database_registry' table as that is what we are trying to populate. 
tables = [table for table in all_tables if table not in tables]

# Iterate through every table.
for table in tables:
    table_name = table[0]
    
    # Query to count the number of rows in the table.
    count_rows_query = f"SELECT COUNT(*) FROM {table_name}"
    mycursor.execute(count_rows_query)
    num_rows = mycursor.fetchone()[0]
    
    # Query to count the number of columns in the table.
    count_columns_query = f"SELECT COUNT(*) FROM information_schema.columns WHERE table_name = '{table_name}'"
    mycursor.execute(count_columns_query)
    num_columns = mycursor.fetchone()[0]

    # Get metadata for the current table from the previously defined dictionairy.
    if table_name in dataset_metadata:
        metadata = dataset_metadata[table_name]
    else:
        metadata = {}  # Set the default to be empty metadata if not found.

    # Insert table information and metadata into the dataset registry table. 
    dataset_registry_query = "INSERT INTO dataset_registry (table_name, num_rows, num_columns, metadata) VALUES (%s, %s, %s, %s)"
    mycursor.execute(dataset_registry_query, (table_name, num_rows, num_columns, json.dumps(metadata)))


mydb.commit()
mydb.close()
